In [4]:
# get_loader

import os
import pandas as pd
import spacy
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import torchvision.transforms as transforms
from konlpy.tag import Okt

#we want to convert text -> numerical values
#1. We need a Vocabulary mapping each word to a index
#2. We need to setup a Pytorch dataset to load the data
#3. Setup padding of every batch (all examples should be)
#   of same seq_len and setup dataloader)

#Download with : python -m spacy download en

Okt = Okt()

class Vocabulary:
    def __init__(self, freq_threshold):
        self.itos = {0:"<PAD>", 1:"<SOS>", 2:"<EOS>", 3:"<UNK>"}
        self.stoi = {"<PAD>":0, "<SOS>":1, "<EOS>":2, "<UNK>":3}
        self.freq_threshold = freq_threshold
        
    def __len__(self):
        return len(self.itos)
    
    @staticmethod
    def tokenizer(text):
        return [tok for tok in Okt.morphs(text)]
    
    #" I love peanuts" -> ["i", "love", :peanuts] ex
    
    def build_vocabulary(self, sentence_list):
        frequencies = {}
        idx = 4
        
        for sentence in sentence_list:
            for word in self.tokenizer(sentence):
                if word not in frequencies:
                    frequencies[word] = 1
                else:
                    frequencies[word] +=1
                    
                if frequencies[word] == self.freq_threshold:
                    self.stoi[word] = idx
                    self.itos[idx] = word
                    idx +=1
                    
    def numericalize(self, text):
        tokenized_text = self.tokenizer(text)
        
        return[
            self.stoi[token] if token in self.stoi else self.stoi["<UNK>"]
            for token in tokenized_text
        ]

    
class FlickrDataset(Dataset): #데이터셋 객체에 들어가는 변수들
    def __init__(self, root_dir, new_captions_final, transform=None, freq_threshold=5):
        self.root_dir = root_dir
        self.df = pd.read_csv(new_captions_final)
        self.transform = transform
        
        #Get img, caption columns
        self.imgs = self.df["image"]
        self.captions = self.df["caption"]
        
        #Initialize vocabulary and build vocab
        self.vocab = Vocabulary(freq_threshold)
        self.vocab.build_vocabulary(self.captions.tolist())
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        caption = self.captions[index]
        img_id = self.imgs[index]
        img = Image.open(os.path.join(self.root_dir, img_id)).convert("RGB")
        
        if self.transform is not None:
            img = self.transform(img)
            
        numericalized_caption = [self.vocab.stoi["<SOS>"]]
        numericalized_caption += self.vocab.numericalize(caption)
        numericalized_caption.append(self.vocab.stoi["<EOS>"])
        
        return img, torch.tensor(numericalized_caption)
def Print(self):
    print(self.captions)
    
class MyCollate:
    def __init__(self, pad_idx):
        self.pad_idx = pad_idx
        
    def __call__(self, batch):
        imgs = [item[0].unsqueeze(0) for item in batch]
        imgs = torch.cat(imgs, dim=0)
        targets = [item[1] for item in batch]
        targets = pad_sequence(targets, batch_first=False, padding_value = self.pad_idx)
        
        return imgs, targets
    
def get_loader(
        root_folder,annotation_file,transform,batch_size = 32,num_workers = 0,shuffle = True,pin_memory = True,):
        dataset = FlickrDataset(root_folder, annotation_file, transform=transform)
    
        pad_idx = dataset.vocab.stoi["<PAD>"]
    
        loader = DataLoader(dataset=dataset,batch_size = batch_size,num_workers = num_workers,shuffle = shuffle,pin_memory = pin_memory,
            collate_fn = MyCollate(pad_idx = pad_idx),)
        return loader,dataset
 




In [76]:
count= 0
for i in dataset.vocab.stoi:
    print(i,count)
    count+=1

<PAD> 0
<SOS> 1
<EOS> 2
<UNK> 3
. 4
개 5
가 6
서로 7
를 8
소녀 9
에 10
어린 11
있습니다 12
에서 13
벤치 14
남자 15
와 16
을 17
모자 18
앞 19
한 20
고 21
의 22
아이 23
로 24
흰 25
검은 26
나무 27
과 28
달리 29
이 30
흰색 31
갈색 32
눈 33
입은 34
속 35
는 36
사람 37
등반 38
밧줄 39
암벽 40
근처 41
잔디 42
하얀 43
공 44
하는 45
합니다 46
함께 47
소년 48
으로 49
옆 50
뛰어 51
개는 52
물 53
호수 54
두 55
해변 56
얼음 57
가지 58
장난감 59
들고 60
있는 61
노란색 62
파란색 63
통해 64
색 65
입 66
빨간색 67
가진 68
그 69
하키 70
앉아 71
큰 72
건물 73
위 74
하고 75
놀고 76
풀밭 77
동안 78
세 79
마리 80
서 81
빨간 82
은 83
앉아있는 84
녹색 85
카약 86
다른 87
여자 88
위해 89
손 90
셔츠 91
벽 92
들 93
스케이트 94
보는 95
보드 96
점프 97
땋 98
머리 99
명의 100
베개 101
다 102
놀아요 103
금발 104
기차 105
보고 106
노 107
향 108
높은 109
다이빙 110
작은 111
따라 112
언덕 113
스키 114
내려다보고 115
어 116
흑인 117
백인 118
오르고 119
거리 120
모래 121
잡기 122
잔디밭 123
잡으려고 124
풍선 125
쓴 126
타고 127
자전거 128
탄다 129
무리 130
밤 131
놀고있다 132
반바지 133
덮인 134
어린이 135
뒤 136
수영장 137
아래 138
공중 139
물속 140
해 141
카메라 142
서있다 143
불 144
얼굴 145
바닥 146
들판 147
야외 148
여러 149
서있는 150
봅니다 151
젊은 152
여성 153
재킷 154
산 155
오르는 156
된 157
달리는 15

야자수 1308
늘어선 1309
공격 1310
너무 1311
헤드 1312
자유 1313
어머니 1314
레슬링 1315
디스크 1316
휴대폰 1317
흔들고 1318
플로 1319
설치 1320
세트 1321
지나가고있다 1322
바람 1323
체인 1324
공간 1325
돈 1326
염소 1327
안고있다 1328
게 1329
마칭 1330
플루트 1331
햇볕 1332
운동 1333
상단 1334
묶여 1335
경쟁 1336
닭 1337
쫓아 1338
다닌다 1339
렉 1340
수건 1341
선택 1342
분필 1343
격자 1344
젓는다 1345
걸 1346
공연 1347
검정 1348
빨강 1349
다시 1350
축구장 1351
떨어져 1352
번 1353
주 1354
낚싯줄 1355
황혼 1356
라켓 1357
설원 1358
바구니 1359
왕관 1360
가짜 1361
화장지 1362
니트 1363
도베르만 1364
싸 1365
팔로 1366
놀면서 1367
견 1368
모퉁이 1369
돌고 1370
망토 1371
덮개 1372
청소년 1373
개가있는 1374
엄지 1375
신사 1376
안전모 1377
지나가고 1378
지상 1379
구를 1380
매달린 1381
소가 1382
기저귀 1383
좁은 1384
극장 1385
가려고 1386
빗속 1387
전자 1388
저지를 1389
낙서 1390
기 1391
플레이 1392
피 1393
숫자 1394
금 1395
헤드셋 1396
코치 1397
태클 1398
상대 1399
리더 1400
치어 1401
도중 1402
쿼터백 1403
대해 1404
검사 1405
Sooners 1406
노력 1407
팬 1408
마스코트 1409
오클라호마 1410
응원 1411
TV 1412
계 1413
미국인 1414
미식 1415
대학 1416
고합 1417
움직입니다 1418
되고 1419
심판 1420
스냅 1421
던질 1422
번호 1423
스트립 1424
퍼레이드 1425
플 1426
춤추는 1427

데스크 2731
노두 2732
체스 2733
민소매 2734
핀 2735
훑어 2736
붙어 2737
독특한 2738
도보 2739
격퇴 2740
고드름 2741
설상차 2742
중형 2743
내려가 2744
다가갑니다 2745
굽히고 2746
터번 2747
모이는 2748
소방차 2749
대를 2750
모이 2751
잠든 2752
백악관 2753
엽 2754
덩크 2755
회의 2756
하강 2757
쓰려고 2758
물개 2759
윤곽 2760
보안 2761
파랑 2762
꿀벌 2763
녹는 2764
전선 2765
일곱 2766
이마 2767
캐치 2768
개보 2769
움직 2770
낮게 2771
모토 2772
힐 2773
아빠 2774
아파트 2775
그려져 2776
장미 2777
하의 2778
밀고있는 2779
스파이크 2780
헬리콥터 2781
카 2782
투우사 2783
나가고 2784
돕기 2785
앉고 2786
매달린다 2787
갔다 2788
센터 2789
보스턴 2790
리그 2791
구역 2792
만들어진 2793
구르는 2794
둘러싸고 2795
선원 2796
민들레 2797
붙잡고있는 2798
액자 2799
미끄러진다 2800
밴 2801
다가 2802
주스 2803
극단 2804
닮은 2805
사리 2806
트로피 2807
양털 2808
가벼운 2809
검색 2810
투명한 2811
가려진 2812
녹슨 2813
강의 2814
캐노 2815
딩 2816
법 2817
걸으며 2818
걷어차는 2819
보호대 2820
사격장 2821
러셀 2822
편 2823
가기 2824
로드 2825
대포 2826
즐긴다 2827
포도 2828
않도록 2829
너 2830
배꼽 2831
과정 2832
펀치 2833
둘러 2834
열고 2835
품 2836
덮는 2837
올린다 2838
줄이 2839
휘두르고 2840
피치 2841
심하게 2842
전동 2843
쓰다듬고 2844
군복 2845
병사 2846
기동 2847
디지털 2848
관 2849
날아

In [33]:
dataset.df[dataset.df['image']=='109738916_236dc456ac.jpg']['caption']

450                          사람들은 스노 모빌을 타고 돌아 다니고 있습니다.
451               사람들은 스노 모빌을 타는 동안 헬멧과 고글을 쓰고 포즈를 취합니다.
452                    스노 모빌을 타는 동안 여러 사람이 휴식을 취하고 있습니다.
453                                    세 사람과 두 대의 스노 모빌.
454    헬멧을 쓴 두 명의 남자가 노란색 스노 모빌에 앉아 있고 다른 한 남자가 지켜보고 ...
Name: caption, dtype: object

In [2]:
#models
import torch
import torch.nn as nn
import statistics
import torchvision.models as models


class EncoderCNN(nn.Module):
    def __init__(self, embed_size, train_CNN=False):
        super(EncoderCNN, self).__init__()
        self.train_CNN = train_CNN
        self.inception = models.inception_v3(pretrained=True, aux_logits=False)
        self.inception.fc = nn.Linear(self.inception.fc.in_features, embed_size)
        self.relu = nn.ReLU()
        self.times = []
        self.dropout = nn.Dropout(0.5)

    def forward(self, images):
        features = self.inception(images)
        return self.dropout(self.relu(features))


class DecoderRNN(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers):
        super(DecoderRNN, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.gru = nn.GRU(embed_size, hidden_size, num_layers)
        self.linear = nn.Linear(hidden_size, vocab_size)
        self.dropout = nn.Dropout(0.3)

    def forward(self, features, captions):
        embeddings = self.dropout(self.embed(captions))
        embeddings = torch.cat((features.unsqueeze(0), embeddings), dim=0)
        hiddens, _ = self.gru(embeddings)
        outputs = self.linear(hiddens)
        return outputs


class CNNtoRNN(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers):
        super(CNNtoRNN, self).__init__()
        self.encoderCNN = EncoderCNN(embed_size)
        self.decoderRNN = DecoderRNN(embed_size, hidden_size, vocab_size, num_layers)

    def forward(self, images, captions):
        features = self.encoderCNN(images)
        outputs = self.decoderRNN(features, captions)
        return outputs

    def caption_image(self, image, vocabulary, max_length=50):
        result_caption = []

        with torch.no_grad():
            x = self.encoderCNN(image).unsqueeze(0)
            states = None

            for _ in range(max_length):
                hiddens, states = self.decoderRNN.gru(x, states)
                output = self.decoderRNN.linear(hiddens.squeeze(0))
                predicted = output.argmax(1)
                result_caption.append(predicted.item())
                x = self.decoderRNN.embed(predicted).unsqueeze(0)
                
                if vocabulary.itos[predicted.item()] == "<EOS>":
                    break

        return [vocabulary.itos[idx] for idx in result_caption]

In [1]:
#utils
import pandas as pd
import torch
import torchvision.transforms as transforms
from PIL import Image
from torchtext.data.metrics import bleu_score
import nltk.translate.bleu_score as bleu
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import corpus_bleu
from datetime import datetime
from konlpy.tag import Okt

import numpy as np
global gram_1
global gram_2
global gram_3
global gram_mean
gram_1=[]
gram_2=[]
gram_3=[]
gram_mean=[]
Okt = Okt()

def print_examples(model, device, dataset):
    img_names=['109671650_f7bbc297fa.jpg','109738763_90541ef30d.jpg','109738916_236dc456ac.jpg',
               '109823394_83fcb735e1.jpg','111537217_082a4ba060.jpg']
    
    img_names=['109671650_f7bbc297fa.jpg']
    
    transform = transforms.Compose(
        [
            transforms.Resize((299, 299)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ]
    )
    
    model.eval()
    for i in img_names:
        if(i=='109738763_90541ef30d.jpg'):
            break;
        test_img1 = transform(Image.open("test_examples/{}".format(i)).convert("RGB")).unsqueeze(0)
        print("Example 1 CORRECT: \n")
        candidate_corpus=dataset.df[dataset.df['image']==i]['caption']
        print(candidate_corpus)
        print("Example 1 OUTPUT: "+ " ".join(model.caption_image(test_img1.to(device), dataset.vocab)[1:-1]))
        c=[]
        for i in img_names:
            candidate_corpus=dataset.df[dataset.df['image']==i]['caption']

            a=np.array(candidate_corpus)
           
            for i in a:
                b=[tok for tok in Okt.morphs(i)]
                c.append(b)
            #bleu_score(c,
        bleu_1gram = sentence_bleu(c, model.caption_image(test_img1.to(device), dataset.vocab)[1:-1], weights=(1, 0, 0, 0))
        bleu_2gram = sentence_bleu(c, model.caption_image(test_img1.to(device), dataset.vocab)[1:-1], weights=(0, 1, 0, 0))
        bleu_3gram = sentence_bleu(c, model.caption_image(test_img1.to(device), dataset.vocab)[1:-1], weights=(0, 0, 1, 0))
        bleu_cum1 = sentence_bleu(c, model.caption_image(test_img1.to(device), dataset.vocab)[1:-1], weights=(0.33, 0.33, 0.33, 0)) #4그램 제외 모두곱하는 특성상 0이나옴
        score = sentence_bleu(c, model.caption_image(test_img1.to(device), dataset.vocab)[1:-1])
        
        print(c)
        print(model.caption_image(test_img1.to(device), dataset.vocab)[1:-1])
    
        print("dsafsdafad score",score)
        print(f'1-Gram BLEU: {bleu_1gram:.2f}')
        print(f'2-Gram BLEU: {bleu_2gram:.2f}')
        print(f'3-Gram BLEU: {bleu_3gram:.2f}')
        
        print(f'Geometric Mean 3-Gram Cumulative BLEU (nltk) : {bleu_cum1:.2f}')

        gram_1.append(f'{bleu_1gram:.2f}')
        gram_2.append(f'{bleu_2gram:.2f}')
        gram_3.append(f'{bleu_3gram:.2f}')
        gram_mean.append(f'{bleu_cum1:.2f}')
        
        d={'1_gram': gram_1,'2_gram':gram_2,'3_gram':gram_3,'1~3_grams_mean':gram_mean}
        df=pd.DataFrame(d)
        df.to_csv("Second_Bleu_graph1.csv",encoding="cp949",index=False)
        print("=> Saving graph.CSV")
    model.train()

def print_caption(model, device, dataset,img_name):
    transform = transforms.Compose(
        [
            transforms.Resize((299, 299)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ]
    )
    now= datetime.now()
    model.eval()
    test_img1 = transform(Image.open("image_input/{}".format(img_name)).convert("RGB")).unsqueeze(0)
    dt_string = f"{now.year}년{now.month}월 {now.day}일 {now.hour:02}시 {now.minute:02}분 {now.second:02}초"
    result=" 현재위치: 손민성 집의 CCTV \n 촬영시각: {} \n 상세내용: ".format(dt_string)+"".join(model.caption_image(test_img1.to(device), dataset.vocab)[1:-1])
    r = open("text_output/text_output.txt", "w")
    r.write(result)
    r.close()
    model.train()

def save_checkpoint(state, filename="my_checkpoint_Adam_GRU_seq2seq1.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)

    


def load_checkpoint(checkpoint, model, optimizer):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])
    step = checkpoint["step"]
    return step

In [ ]:
#training
import torch
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
import konlpy
import pickle
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import corpus_bleu

def train():
    #loss_list=pd.read_csv('loss_list.csv')
    a=[]
    #a=np.array(loss_list['loss'])
    transform = transforms.Compose(
        [
            transforms.Resize((356, 356)),
            transforms.RandomCrop((299, 299)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ]
    )

    train_loader, dataset = get_loader(
        root_folder="Flicker8k_Dataset",
        annotation_file="new_captions_final_최종.txt",
        transform=transform,
        num_workers=0,
    )
    with open('train_loader.pkl', 'wb') as f:
        pickle.dump(train_loader, f)
    
    with open('dataset.pkl', 'wb') as f:
        pickle.dump(dataset, f)

    torch.backends.cudnn.benchmark = True
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    load_model = False
    save_model = True
    train_CNN = False

    # Hyperparameters
    embed_size = 256
    hidden_size = 256
    vocab_size = len(dataset.vocab)
    num_layers = 1
    learning_rate = 3e-4
    num_epochs = 1000

    # for tensorboard
    writer = SummaryWriter("runs/flickr")
    step = 0

    # initialize model, loss etc
    model = CNNtoRNN(embed_size, hidden_size, vocab_size, num_layers).to(device)
    criterion = nn.CrossEntropyLoss(ignore_index=dataset.vocab.stoi["<PAD>"])
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Only finetune the CNN
    for name, param in model.encoderCNN.inception.named_parameters():
        if "fc.weight" in name or "fc.bias" in name:
            param.requires_grad = True
        else:
            param.requires_grad = train_CNN

    if load_model:
        step = load_checkpoint(torch.load("my_checkpoint_Adam_GRU_seq2seq1.pth.tar"), model, optimizer)

    model.train()
    
    for epoch in range(num_epochs):
        # Uncomment the line below to see a couple of test cases
        
        print_examples(model, device, dataset)
        
        if save_model:
            checkpoint = {
                "state_dict": model.state_dict(),
                "optimizer": optimizer.state_dict(),
                "step": step,
            }
            save_checkpoint(checkpoint)
        loss_save=[]
        for idx, (imgs, captions) in tqdm(
            enumerate(train_loader), total=len(train_loader), leave=False
        ):
            imgs = imgs.to(device)
            captions = captions.to(device)

            outputs = model(imgs, captions[:-1])
            loss = criterion(
                outputs.reshape(-1, outputs.shape[2]), captions.reshape(-1)
            )
            writer.add_scalar("Training loss", loss.item(), global_step=step)
            step += 1

            optimizer.zero_grad()
            loss.backward(loss)
            optimizer.step()
            loss_save.append(loss.item())
        loss_ave=np.mean(loss_save)
        save_csv(loss_ave)
    
            
def save_csv(a):
    df=pd.DataFrame({'loss' : a})
    df.to_csv('second_loss_list1.csv')

if __name__ == "__main__":

    train()